In [189]:
import pandas as pd

halifax_weather = pd.read_csv('WeatherData/ColdnHumid/halifax_Weather_Solar.csv')
halifax_weather = halifax_weather.drop(columns=['DHI (w/m2)', 'DNI (w/m2)', 'Pressure (mbar)'])

halifax_weather.head()

,Year,Month,Day,Max Temp (F),Min Temp (F),Wind Speed (m/s),Relative Humidity (%)
0,2015,1,1,28.04,5.18,0.2,75.54
1,2015,1,2,34.34,13.64,0.4,73.16
2,2015,1,3,14.00,5.36,0.4,54.87
3,2015,1,4,46.58,7.88,0.4,86.75
4,2015,1,5,46.76,9.14,0.6,81.97


In [190]:
def ms_to_mph(wind_speed):
    return round(wind_speed * 2.23694, 1)

In [191]:
halifax_weather.insert(loc=5, column='Average Wind Speed (mph)', value=0, allow_duplicates=False)

In [192]:
halifax_weather['Average Wind Speed (mph)'] = halifax_weather.apply(lambda row: ms_to_mph(row['Wind Speed (m/s)']), axis=1)

In [193]:
halifax_weather = halifax_weather.drop(columns=['Wind Speed (m/s)'])
halifax_weather.head()

,Year,Month,Day,Max Temp (F),Min Temp (F),Average Wind Speed (mph),Relative Humidity (%)
0,2015,1,1,28.04,5.18,0.4,75.54
1,2015,1,2,34.34,13.64,0.9,73.16
2,2015,1,3,14.00,5.36,0.9,54.87
3,2015,1,4,46.58,7.88,0.9,86.75
4,2015,1,5,46.76,9.14,1.3,81.97


In [194]:
halifax_weather['Max Temp (F)'] = halifax_weather.apply(lambda row: round(row['Max Temp (F)']), axis=1)
halifax_weather['Min Temp (F)'] = halifax_weather.apply(lambda row: round(row['Min Temp (F)']), axis=1)
halifax_weather['Relative Humidity (%)'] = halifax_weather.apply(lambda row: round(row['Relative Humidity (%)']), axis=1)

In [195]:
halifax_weather.head()

,Year,Month,Day,Max Temp (F),Min Temp (F),Average Wind Speed (mph),Relative Humidity (%)
0,2015,1,1,28,5,0.4,76
1,2015,1,2,34,14,0.9,73
2,2015,1,3,14,5,0.9,55
3,2015,1,4,47,8,0.9,87
4,2015,1,5,47,9,1.3,82


In [196]:
from math import pow

def windchill_regression(temperature, wind_speed):
    T, V = temperature, wind_speed
    feels_like = 35.74 + (0.6215) * (T) - (35.75) * (pow(V, 0.16)) + (0.4275 * T) * (pow(V, 0.16))
    return round(feels_like)

In [197]:
def get_windchill(temperature, wind_speed):
    T, V = temperature, wind_speed
    if (T > 50):
        return temperature
    else:
        return windchill_regression(T, V)

In [198]:
halifax_weather.insert(loc=7, column='Windchill Effect (F)', value=0, allow_duplicates=False)

In [199]:
halifax_weather['Windchill Effect (F)'] = halifax_weather.apply(lambda row: get_windchill(row['Max Temp (F)'], row['Average Wind Speed (mph)']), axis=1)
halifax_weather['Windchill Effect (F)'] = halifax_weather.apply(lambda row: round(row['Windchill Effect (F)']), axis=1)

In [200]:
halifax_weather.head(20)

,Year,Month,Day,Max Temp (F),Min Temp (F),Average Wind Speed (mph),Relative Humidity (%),Windchill Effect (F)
0,2015,1,1,28,5,0.4,76,33
1,2015,1,2,34,14,0.9,73,36
2,2015,1,3,14,5,0.9,55,15
3,2015,1,4,47,8,0.9,87,50
4,2015,1,5,47,9,1.3,82,49
5,2015,1,6,9,3,1.1,64,9
6,2015,1,7,11,-3,0.7,66,13
7,2015,1,8,8,0,0.9,72,9
8,2015,1,9,38,2,1.1,85,40
9,2015,1,10,22,13,0.9,72,24


In [201]:
halifax_weather.insert(loc=8, column='Users Thermostat (F)', value=0, allow_duplicates=False)

In [202]:
import random
from time import gmtime
from calendar import timegm

def user_heating(windchill_temperature):
    # Gets the systems milliseconds since the epoch
    mseconds = timegm(gmtime())
    # Sets the milliseconds as the random seed for generation
    rand_seed = random.seed(mseconds * random.random())

    min_heat = 60
    avg_heat = 66
    max_heat = 71
    if (windchill_temperature <= 45):
        return random.randrange(avg_heat, max_heat, 1)
    elif (windchill_temperature > 45):
        return random.randrange(min_heat, avg_heat, 1)

def user_cooling(feels_like_temperature):
    # Gets the systems milliseconds since the epoch
    mseconds = timegm(gmtime())
    # Sets the milliseconds as the random seed for generation
    rand_seed = random.seed(mseconds * random.random())

    min_cool = 74
    avg_cool = 79
    max_cool = 85
    if (feels_like_temperature >= 90):
        return random.randrange(min_cool, avg_cool, 1)
    elif (feels_like_temperature < 90):
        return random.randrange(avg_cool, max_cool, 1)

def user_normal(temperature):
    return floor(temperature)

def get_user_data(weighted_temperature):
    if (weighted_temperature <= 64):
        # The WHO recommends a minimum indoor temperature of 64 degrees Fahrenheit
        return user_heating(windchill_temperature=weighted_temperature) # Run the heat
    elif (weighted_temperature > 64 and weighted_temperature <= 80):
        return user_normal(weighted_temperature) # The house is at room temperature
    elif (weighted_temperature > 80):
        return user_cooling(feels_like_temperature=weighted_temperature) # Otherwise run the A/C

In [203]:
halifax_weather['Users Thermostat (F)'] = halifax_weather.apply(lambda row: get_user_data(row['Windchill Effect (F)']), axis=1)

In [204]:
halifax_weather.tail(30)

,Year,Month,Day,Max Temp (F),Min Temp (F),Average Wind Speed (mph),Relative Humidity (%),Windchill Effect (F),Users Thermostat (F)
1795,2019,12,2,32,18,1.8,78,31,70
1796,2019,12,3,43,31,1.8,95,43,66
1797,2019,12,4,39,27,1.6,95,39,69
1798,2019,12,5,36,27,0.9,95,38,70
1799,2019,12,6,33,26,1.8,86,32,68
1800,2019,12,7,27,16,1.8,71,26,68
1801,2019,12,8,30,15,1.8,80,29,68
1802,2019,12,9,51,30,2.0,94,51,65
1803,2019,12,10,55,51,2.9,99,55,65
1804,2019,12,11,51,20,1.6,90,51,61


In [206]:
halifax_weather.to_csv(path_or_buf='WeatherData/ColdnHumid/halifax/halifaxCompleteUser.csv')